In [1]:
import pandas as pd
import numpy as np
import scipy as sp

In [2]:
import os, sys

In [3]:
print( os.curdir )
print( os.getcwd() )
print( os.path.abspath( os.getcwd() ) )

.
/Users/nururrahman/myRecSystem
/Users/nururrahman/myRecSystem


### Transaction Data

In [4]:
date  = ['day1','day1','day1','day1','day1','day2','day2','day2','day2','day2']
users = ['u1','u1','u1','u2','u2','u1','u3','u3','u3','u3']
items = ['item1','item2','item4','item2','item3','item2','item1','item3','item4','item3']
transaction = pd.DataFrame({'date':date, 'user':users, 'item':items})
transaction

,date,user,item
0,day1,u1,item1
1,day1,u1,item2
2,day1,u1,item4
3,day1,u2,item2
4,day1,u2,item3
5,day2,u1,item2
6,day2,u3,item1
7,day2,u3,item3
8,day2,u3,item4
9,day2,u3,item3


#### Add count

In [5]:
trans_count = transaction.groupby(['user','item']).size().reset_index().rename(columns={0:'count'})
trans_count

,user,item,count
0,u1,item1,1
1,u1,item2,2
2,u1,item4,1
3,u2,item2,1
4,u2,item3,1
5,u3,item1,1
6,u3,item3,2
7,u3,item4,1


### Pivot

In [6]:
user_item = trans_count.pivot(index='user',columns='item', values='count')
user_item.index.name = ''
user_item.columns.name = ''
user_item = user_item.fillna(value=0, inplace=False)
user_item = user_item.astype('int8')
user_item

,item1,item2,item3,item4
,,,,
u1,1,2,0,1
u2,0,1,1,0
u3,1,0,2,1


### Melt

In [7]:
data = user_item.reset_index(drop=False).rename(columns={'':'user'})

In [8]:
transaction = data.melt(
    id_vars = 'user', 
    value_vars = data.columns[1:], 
    var_name = 'item', 
    value_name = 'count'
)

transaction

,user,item,count
0,u1,item1,1
1,u2,item1,0
2,u3,item1,1
3,u1,item2,2
4,u2,item2,1
5,u3,item2,0
6,u1,item3,0
7,u2,item3,1
8,u3,item3,2
9,u1,item4,1


### Co-Occurance Matrix

In [9]:
clipped = user_item.clip(lower=0, upper=1, inplace=False)
clipped

,item1,item2,item3,item4
,,,,
u1,1,1,0,1
u2,0,1,1,0
u3,1,0,1,1


In [10]:
cooc = clipped.T.dot(clipped)
print( cooc ) 

       item1  item2  item3  item4
                                 
item1      2      1      1      2
item2      1      2      1      1
item3      1      1      2      1
item4      2      1      1      2


In [11]:
np.fill_diagonal(cooc.values, 0)
print( cooc ) 

       item1  item2  item3  item4
                                 
item1      0      1      1      2
item2      1      0      1      1
item3      1      1      0      1
item4      2      1      1      0


In [12]:
user_item.clip(lower=0, upper=1, inplace=False)

,item1,item2,item3,item4
,,,,
u1,1,1,0,1
u2,0,1,1,0
u3,1,0,1,1
